<a href="https://colab.research.google.com/github/JoyeBright/Semi-supervised-sentiment/blob/ImageLadder/ladder_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tqdm import tqdm
%run /content/ladder_input.ipynb

In [50]:
layer_sizes = [784, 1000, 500, 250, 250, 250, 10]

L = len(layer_sizes) - 1  # number of layers

num_examples = 60000 # a training set of 60,000 examples
num_epochs = 150
num_labeled = 100

started_learning_rate = 0.02

decay_after = 15

batch_size = 100

num_iter = (num_examples/batch_size) * num_epochs



6
